<a href="https://www.kaggle.com/code/dascient/short-squeeze-signals-donutz-ai?scriptVersionId=298311786" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#!/usr/bin/env python
# Enhanced Real-Time Trading Signal Script for 10,000+ Tickers
# ---------------------------------------------------------------
# This script downloads 1-minute intraday stock data for a large universe of tickers,
# calculates an expanded set of technical indicators, and evaluates a composite scoring system.
# With adjustable thresholds and dynamic risk management (ATR-based), it is designed for real-time use.
#
# Key Improvements:
#   - Adjustable composite thresholds for increased sensitivity (default: 3/7).
#   - Batching and high-parallelism processing to handle up to 10,000 tickers.
#   - Scheduled real-time runs (every minute during market hours).
#   - Enhanced logging and error handling for reliability.
#
# DISCLAIMER: This script is for educational purposes only and does not constitute financial advice.
# Always perform your own due diligence and risk management before trading.

!pip install yfinance pandas tqdm schedule config --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import concurrent.futures
import time
import schedule
from tqdm import tqdm

# Global verbosity flag.
verbose = True

# Adjustable thresholds for composite scoring (lower = more sensitive)
COMPOSITE_LONG_THRESHOLD = 3  # out of 7
COMPOSITE_SHORT_THRESHOLD = 3  # out of 7

# Attempt GPU acceleration using cuDF if available.
try:
    import cudf
    gpu_enabled = True
    if verbose:
        print("GPU acceleration enabled using cuDF.")
except ImportError:
    gpu_enabled = False
    if verbose:
        print("cuDF not available; using pandas.")

def get_stock_data(ticker, period='1d', interval='1m'):
    """Download intraday stock data from yfinance for a single ticker."""
    if verbose:
        print(f"[{ticker}] Downloading {interval} data for period: {period}...")
    try:
        data = yf.download(ticker, period=period, interval=interval, progress=False)
        if verbose:
            print(f"[{ticker}] Downloaded {len(data)} rows.")
        return data
    except Exception as e:
        print(f"[{ticker}] Error downloading data: {e}")
        return pd.DataFrame()

def calculate_indicators(df):
    """
    Calculate advanced technical indicators for the dataframe:
      - EMAs: 20, 50, 100
      - RSI (14)
      - MACD & Signal
      - Bollinger Bands (20)
      - ATR (14)
      - ADX (14)
      - Stochastic Oscillator (14)
      - VWAP (intraday)
      - Volume MA (20)
    """
    if verbose:
        print("Calculating advanced indicators...")
    if gpu_enabled:
        df = cudf.DataFrame.from_pandas(df)
    
    # EMAs
    df['EMA20'] = df['Adj Close'].ewm(span=20, adjust=False).mean()
    df['EMA50'] = df['Adj Close'].ewm(span=50, adjust=False).mean()
    df['EMA100'] = df['Adj Close'].ewm(span=100, adjust=False).mean()

    # RSI (14)
    delta = df['Adj Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD & Signal
    ema12 = df['Adj Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Adj Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands (20)
    df['BB_Middle'] = df['Adj Close'].rolling(window=20).mean()
    rolling_std = df['Adj Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (2 * rolling_std)
    df['BB_Lower'] = df['BB_Middle'] - (2 * rolling_std)

    # ATR (14)
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Adj Close'].shift())
    df['Low-Close'] = np.abs(df['Low'] - df['Adj Close'].shift())
    df['TR'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=14).mean()

    # ADX (14)
    df['UpMove'] = df['High'] - df['High'].shift()
    df['DownMove'] = df['Low'].shift() - df['Low']
    df['+DM'] = np.where((df['UpMove'] > df['DownMove']) & (df['UpMove'] > 0), df['UpMove'], 0.0)
    df['-DM'] = np.where((df['DownMove'] > df['UpMove']) & (df['DownMove'] > 0), df['DownMove'], 0.0)
    df['+DI'] = 100 * (df['+DM'].rolling(window=14).sum() / df['ATR'])
    df['-DI'] = 100 * (df['-DM'].rolling(window=14).sum() / df['ATR'])
    df['DX'] = (np.abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].rolling(window=14).mean()

    # Stochastic Oscillator (14)
    df['Lowest_14'] = df['Low'].rolling(window=14).min()
    df['Highest_14'] = df['High'].rolling(window=14).max()
    df['%K'] = 100 * ((df['Adj Close'] - df['Lowest_14']) / (df['Highest_14'] - df['Lowest_14']))
    df['%D'] = df['%K'].rolling(window=3).mean()

    # VWAP (Volume Weighted Average Price)
    df['Cum_Volume'] = df['Volume'].cumsum()
    df['Cum_PV'] = (df['Adj Close'] * df['Volume']).cumsum()
    df['VWAP'] = df['Cum_PV'] / df['Cum_Volume']

    # Volume MA (20)
    df['Volume_MA'] = df['Volume'].rolling(window=20).mean()

    # Clean up intermediate columns
    cols_to_drop = ['High-Low', 'High-Close', 'Low-Close', 'UpMove', 'DownMove', '+DM', '-DM']
    df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    
    if gpu_enabled:
        df = df.to_pandas()

    if verbose:
        print("Advanced indicators calculated.")
    return df

def composite_long_signal(df, volume_multiplier=1.5, adx_threshold=20):
    """
    Evaluate a composite long signal.
    Conditions (each worth 1 point):
      1. Price above both EMA20 & EMA50.
      2. RSI bullish (50 < RSI < 70).
      3. MACD > MACD_Signal.
      4. ADX > adx_threshold.
      5. Stochastic (%K < 80 & %D < 80).
      6. Price above VWAP.
      7. Volume spike: current volume > Volume_MA * volume_multiplier.
    Signal triggers if composite score >= COMPOSITE_LONG_THRESHOLD.
    """
    current_price = df['Adj Close'].iloc[-1]
    score = 0
    # Condition 1
    if current_price > df['EMA20'].iloc[-1] and current_price > df['EMA50'].iloc[-1]:
        score += 1
    # Condition 2
    if 50 < df['RSI'].iloc[-1] < 70:
        score += 1
    # Condition 3
    if df['MACD'].iloc[-1] > df['MACD_Signal'].iloc[-1]:
        score += 1
    # Condition 4
    if df['ADX'].iloc[-1] > adx_threshold:
        score += 1
    # Condition 5
    if (df['%K'].iloc[-1] < 80) and (df['%D'].iloc[-1] < 80):
        score += 1
    # Condition 6
    if current_price > df['VWAP'].iloc[-1]:
        score += 1
    # Condition 7
    if df['Volume'].iloc[-1] > (df['Volume_MA'].iloc[-1] * volume_multiplier):
        score += 1

    if verbose:
        print(f"Composite Long Score: {score} (Threshold: {COMPOSITE_LONG_THRESHOLD})")
    return score >= COMPOSITE_LONG_THRESHOLD

def composite_short_signal(df, volume_multiplier=1.5, adx_threshold=20):
    """
    Evaluate a composite short signal.
    Conditions (each worth 1 point):
      1. Price below both EMA20 & EMA50.
      2. RSI bearish (30 < RSI < 50).
      3. MACD < MACD_Signal.
      4. ADX > adx_threshold.
      5. Stochastic (%K > 20 & %D > 20).
      6. Price below VWAP.
      7. Volume spike.
    Signal triggers if composite score >= COMPOSITE_SHORT_THRESHOLD.
    """
    current_price = df['Adj Close'].iloc[-1]
    score = 0
    # Condition 1
    if current_price < df['EMA20'].iloc[-1] and current_price < df['EMA50'].iloc[-1]:
        score += 1
    # Condition 2
    if 30 < df['RSI'].iloc[-1] < 50:
        score += 1
    # Condition 3
    if df['MACD'].iloc[-1] < df['MACD_Signal'].iloc[-1]:
        score += 1
    # Condition 4
    if df['ADX'].iloc[-1] > adx_threshold:
        score += 1
    # Condition 5
    if (df['%K'].iloc[-1] > 20) and (df['%D'].iloc[-1] > 20):
        score += 1
    # Condition 6
    if current_price < df['VWAP'].iloc[-1]:
        score += 1
    # Condition 7
    if df['Volume'].iloc[-1] > (df['Volume_MA'].iloc[-1] * volume_multiplier):
        score += 1

    if verbose:
        print(f"Composite Short Score: {score} (Threshold: {COMPOSITE_SHORT_THRESHOLD})")
    return score >= COMPOSITE_SHORT_THRESHOLD

def process_ticker(ticker_item):
    """Download, calculate indicators, and evaluate composite signals for one ticker."""
    ticker = ticker_item[0] if isinstance(ticker_item, (list, tuple, np.ndarray)) else ticker_item
    if verbose:
        print(f"\n--- Processing {ticker} ---")
    data = get_stock_data(ticker)
    if data.empty or len(data) < 30:  # Skip if insufficient data
        if verbose:
            print(f"[{ticker}] Insufficient or no data.")
        return None

    try:
        data = calculate_indicators(data)
    except Exception as e:
        print(f"[{ticker}] Error calculating indicators: {e}")
        return None

    signal = {}
    if composite_long_signal(data):
        if verbose:
            print(f"[{ticker}] Enhanced LONG opportunity detected.")
        signal['Long Opportunity'] = True
    if composite_short_signal(data):
        if verbose:
            print(f"[{ticker}] Enhanced SHORT opportunity detected.")
        signal['Short Opportunity'] = True

    if signal:
        current_price = data['Adj Close'].iloc[-1]
        atr = data['ATR'].iloc[-1]
        # Dynamic risk management using ATR
        if 'Long Opportunity' in signal:
            stop_loss = current_price - (2 * atr)
            take_profit = current_price + (3 * atr)
        elif 'Short Opportunity' in signal:
            stop_loss = current_price + (2 * atr)
            take_profit = current_price - (3 * atr)
        else:
            stop_loss, take_profit = None, None

        signal.update({
            'Ticker': ticker,
            'Price': current_price,
            'ATR': atr,
            'Stop Loss': stop_loss,
            'Take Profit': take_profit,
            'RSI': data['RSI'].iloc[-1],
            'MACD': data['MACD'].iloc[-1],
            'MACD_Signal': data['MACD_Signal'].iloc[-1],
            'EMA20': data['EMA20'].iloc[-1],
            'EMA50': data['EMA50'].iloc[-1],
            'VWAP': data['VWAP'].iloc[-1]
        })
        return signal
    else:
        if verbose:
            print(f"[{ticker}] No valid signal.")
    return None

def process_ticker_batch(ticker_batch):
    """Process a batch of tickers concurrently."""
    signals = []
    # Increase max_workers for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        futures = {executor.submit(process_ticker, ticker): ticker for ticker in ticker_batch}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Batch Processing"):
            try:
                result = future.result()
                if result:
                    signals.append(result)
            except Exception as e:
                print("Error in batch:", e)
    return signals

def identify_trading_opportunities(tickers):
    """Identify trading opportunities across all tickers by batching."""
    all_signals = []
    batch_size = 50  # adjust batch size as needed
    for i in range(0, len(tickers), batch_size):
        ticker_batch = tickers[i:i+batch_size]
        if verbose:
            print(f"\nProcessing batch {i // batch_size + 1} of {len(tickers) // batch_size + 1}")
        batch_signals = process_ticker_batch(ticker_batch)
        all_signals.extend(batch_signals)
    return all_signals

def is_market_open():
    """Check if the market is open (example for NYSE hours, adjust as needed)."""
    now = datetime.datetime.now()
    return now.weekday() < 5 and (9 <= now.hour < 16)

def run_trading_scan():
    """Run the trading opportunity scan if the market is open."""
    if not is_market_open():
        print("Market is closed. Waiting for next run...")
        return

    # Read tickers from CSV files (update paths as needed)
    try:
        tickers_amex = pd.read_csv("/kaggle/input/tickers-stock-market-amex/tickers.csv", header=None).values.tolist()
        tickers_info = pd.read_csv("/kaggle/input/tickers-stocks/stock_info.csv", header=None).values.tolist()
    except Exception as e:
        print("Error reading ticker CSV files:", e)
        return

    combined_tickers = tickers_amex + tickers_info
    # Remove duplicates
    combined_tickers = list({tuple(ticker) for ticker in combined_tickers})
    combined_tickers = [list(ticker) for ticker in combined_tickers]
    print(f"Total unique tickers: {len(combined_tickers)}")

    trading_signals = identify_trading_opportunities(combined_tickers)

    long_signals = [sig for sig in trading_signals if sig.get('Long Opportunity')]
    short_signals = [sig for sig in trading_signals if sig.get('Short Opportunity')]

    print("\n--- Trading Signal Results ---")
    if long_signals:
        df_long = pd.DataFrame(long_signals)
        print("\nEnhanced Long Opportunities Detected:")
        print(df_long.head())
    else:
        print("\nNo enhanced long opportunities detected.")

    if short_signals:
        df_short = pd.DataFrame(short_signals)
        print("\nEnhanced Short Opportunities Detected:")
        print(df_short.head())
    else:
        print("\nNo enhanced short opportunities detected.")

    print("\nDisclaimer: This script is for educational purposes only and does not constitute financial advice.")

# Schedule the scan to run every minute during market hours.
schedule.every(1).minutes.do(run_trading_scan)

if __name__ == "__main__":
    print("Starting real-time trading scan. Press Ctrl+C to stop.")
    while True:
        schedule.run_pending()
        time.sleep(1)

GPU acceleration enabled using cuDF.
Starting real-time trading scan. Press Ctrl+C to stop.
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting for next run...
Market is closed. Waiting 

In [ ]:
# en fin